In [1]:
from assets.data_generator import DR_Generator
import os
from assets.models import SMD_Unet
# from assets.trainer import Trainer
from assets.test.trainer import Trainer
import tensorflow as tf

In [2]:
masks = ['HardExudate_Masks', 'Hemohedge_Masks', 'Microaneurysms_Masks', 'SoftExudate_Masks']
mask_dir = '../data/FGADR/Seg-set'
mask_paths = [os.path.join(mask_dir, mask) for mask in masks]

generator_args = {
    'dir_path':'../data/FGADR/Seg-set/Original_Images/',
    'mask_path':mask_paths,
    'use_mask':True,
    'img_size':(512, 512),  
    'batch_size':4, # 8로 하면 바로 OOM 뜸
    'dataset':'FGADR', # FGADR or EyePacks
    'use_3channel':False,
    'CLAHE_args':None,
    'add_noise_std':None
}

tr_fgadr_gen = DR_Generator(start_end_index=(0, 1292), is_train=True, **generator_args)
val_fgadr_gen = DR_Generator(start_end_index=(1292, 1476), is_train=False, **generator_args)
ts_fgadr_gen = DR_Generator(start_end_index=(1476, 1842), is_train=False, **generator_args)

# tr_fgadr_gen = DR_Generator(start_end_index=(0, 30000), is_train=True, **generator_args)
# val_fgadr_gen = DR_Generator(start_end_index=(30000, 31000), is_train=False, **generator_args)

In [3]:
model = SMD_Unet(enc_filters=[64, 128, 256, 512, 1024], dec_filters=[512, 256, 64, 32], input_channel=1)

In [4]:
model.load_weights('../models/withoutCLAHE_pretrained_1channel')

In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1) 

trainer_args = {
    'model':model,
    'epochs':30,
    'optimizer':optimizer,
    'for_recons':False,
    'alpha':0.1,
    'beta':None,
    'first_epoch':1,
    'file_name':'history/withoutCLAHE_pretrained_1channel_lr01_alpa01.txt',
    'save_model_path':'../models/withoutCLAHE_pretrained_1channel_lr01_alpa01',
    'add_noise':False
}
trainer = Trainer(**trainer_args)

grads = trainer.train(train_dataset=tr_fgadr_gen,
                        val_dataset=val_fgadr_gen,
                       )


Epoch 1/30
[<tf.Tensor: shape=(3, 3, 1, 64), dtype=float32, numpy=
array([[[[ 6.96723248e-08,  4.75165933e-07,  3.35577056e-07,
           5.13326256e-07,  3.47733931e-07,  8.84006852e-07,
           2.45079974e-08,  1.67552159e-07,  2.07214121e-08,
           4.10796275e-07, -2.99218755e-07,  2.66418596e-07,
           5.25668256e-08,  4.24686249e-07,  2.65237844e-07,
           3.15075880e-07,  4.48404023e-07, -5.49689162e-07,
           4.27065629e-07, -1.85239429e-07, -4.57578082e-08,
           4.76320338e-07,  6.23467201e-07, -3.30262992e-07,
           9.20874612e-08,  0.00000000e+00,  2.52263646e-07,
           1.70494133e-07,  3.64161679e-07, -1.69784428e-07,
          -2.31592239e-08, -2.78368134e-07,  6.79668460e-07,
           3.00843681e-07,  7.09650692e-07,  1.92268487e-07,
           3.85893458e-07, -3.58120872e-07,  9.66786615e-08,
          -6.57995145e-07, -4.05460526e-07,  4.42320982e-07,
          -1.07060188e-07,  1.24410107e-07,  2.94399655e-07,
           3.7997

   4/1292 [..............................] - ETA: 1:44:46 - train_loss: 0.8991 - mask_loss: 0.9990 - loss_recons: 1.4326e-06[<tf.Tensor: shape=(3, 3, 1, 64), dtype=float32, numpy=
array([[[[ 7.35466941e-08,  0.00000000e+00,  1.17612956e-14,
           3.03006753e-09, -2.48527385e-08, -9.41738136e-08,
           6.03258199e-12,  8.16275758e-09,  3.49604156e-12,
           1.87490006e-11,  2.65702766e-07,  1.11269910e-10,
           2.08519367e-07, -1.20095081e-07,  1.53031777e-07,
           1.35605638e-10,  3.04192298e-08,  1.82519486e-07,
           4.87359948e-08,  3.93859034e-09,  2.46270332e-07,
           2.49436383e-08, -3.00134850e-08,  2.84413233e-07,
           1.95162926e-07,  0.00000000e+00,  6.03995354e-10,
           1.51413971e-07,  1.46157122e-08,  3.46948269e-07,
           7.18847470e-09,  2.63900830e-07,  0.00000000e+00,
           5.90264335e-08,  1.08742789e-07, -5.74533381e-08,
           1.18901092e-07,  1.30193627e-07,  4.84587717e-12,
           2.32217957e-07, 

   8/1292 [..............................] - ETA: 58:02 - train_loss: 0.9116 - mask_loss: 0.9985 - loss_recons: 0.1293      [<tf.Tensor: shape=(3, 3, 1, 64), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

KeyboardInterrupt: 